필요한 라이브러리 설치

In [1]:
!pip install qdrant-client sentence-transformers pandas rich

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.3/337.3 kB 4.1 MB/s eta 0:00:00


## 2단계: 데이터 로드 및 전처리

In [3]:
import pandas as pd
from tqdm.auto import tqdm

# 1. CSV 파일 로드
# Colab에 파일을 업로드한 후 파일 경로를 확인하세요.
file_path = 'gov24_services_with_tags (1).csv'
df = pd.read_csv(file_path)

# 2. 벡터화를 위한 텍스트 생성
# NaN(결측치) 값을 안전하게 처리하기 위해 fillna('')를 사용합니다.
df['combined_text'] = (
    (df['서비스명'].fillna('') + " " + df['tags'].fillna('')) * 3 + # 중요 정보를 3번 반복
    " " + df['지원내용'].fillna('')
)

# 3. 페이로드(Payload) 데이터 준비
# 각 행을 dictionary 형태로 변환하여 리스트에 저장합니다.
# 이렇게 하면 각 서비스의 모든 정보를 페이로드로 저장할 수 있습니다.
payloads = df.to_dict(orient='records')

# 4. 벡터화할 텍스트 리스트 준비
documents = df['combined_text'].tolist()

print(f"총 {len(documents)}개의 서비스를 처리합니다.")
print("벡터화 예시 텍스트:", documents[0])

총 300개의 서비스를 처리합니다.
벡터화 예시 텍스트: 유아학비 (누리과정) 지원 ['유아학비', '저소득층', '유치원', '교육비', '지원대상', '유아', '사립유치원']유아학비 (누리과정) 지원 ['유아학비', '저소득층', '유치원', '교육비', '지원대상', '유아', '사립유치원']유아학비 (누리과정) 지원 ['유아학비', '저소득층', '유치원', '교육비', '지원대상', '유아', '사립유치원'] ○ 3~5세에 대해 교육비를 지급합니다.
- 국공립 100,000원, 사립 280,000원
○ 3~5세에 대해 방과후과정비를 지급합니다.
- 국공립 50,000원, 사립 70,000원
○ 사립유치원을 다니는 법정저소득층 유아에게 저소득층 유아학비를 추가 지급합니다.
- 사립 200,000원


## 3단계: 모델 및 Qdrant 클라이언트 준비

In [4]:
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient, models

# 1. 한국어 특화 임베딩 모델 로드
model = SentenceTransformer('jhgan/ko-sroberta-multitask')

# 모델의 벡터 차원 확인 (ko-sroberta-multitask는 768)
vector_size = model.get_sentence_embedding_dimension()
print(f"사용할 모델의 벡터 차원: {vector_size}")

# 2. Qdrant 클라이언트 초기화 (In-memory storage)
client = QdrantClient(":memory:")

# 3. Qdrant 컬렉션 생성
collection_name = "gov_services"
client.recreate_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(size=vector_size, distance=models.Distance.COSINE)
)
print(f"'{collection_name}' 컬렉션 생성 완료!")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/442M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

사용할 모델의 벡터 차원: 768
'gov_services' 컬렉션 생성 완료!


/tmp/ipython-input-228796018.py:16: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  client.recreate_collection(


## 4단계: 벡터 생성 및 Qdrant에 업로드

In [5]:
# 텍스트를 벡터로 변환 (시간이 다소 소요될 수 있습니다)
print("임베딩 생성 중...")
vectors = model.encode(documents, show_progress_bar=True)

# Qdrant에 데이터 포인트 업로드
print("Qdrant에 데이터 업로드 중...")
client.upload_points(
    collection_name=collection_name,
    points=[
        models.PointStruct(
            id=idx,
            vector=vector.tolist(),
            payload=payload
        )
        for idx, (vector, payload) in enumerate(zip(vectors, payloads))
    ],
    batch_size=256, # 한 번에 업로드할 데이터 수
    wait=True
)

print("업로드 완료!")

임베딩 생성 중...


Batches:   0%|          | 0/10 [00:00<?, ?it/s]

Qdrant에 데이터 업로드 중...
업로드 완료!


## 5단계: 의미 기반 검색 테스트

In [7]:
from rich.console import Console
from rich.table import Table
import re # 태그를 파싱하기 위해 정규식 라이브러리 import

def search_services(query_text, top_k=3):
    """(최종 개선) 필드별 가중치를 적용한 정교한 Reranking 검색 함수"""

    # 1. Qdrant에서 후보군 넉넉하게 가져오기 (limit=10)
    response = client.query_points(
        collection_name=collection_name,
        query=model.encode(query_text).tolist(),
        limit=10
    )
    hits = response.points

    # 2. 정교한 Reranking 수행
    reranked_hits = []
    query_keywords = query_text.split()

    for hit in hits:
        payload = hit.payload
        bonus = 0

        # --- 필드별 가중치 차등 적용 ---

        # 1. Tags 필드: 가장 높은 가중치 (+0.5)
        # "노인,일자리,건강" 형태의 태그 문자열을 리스트로 변환 -> ['노인', '일자리', '건강']
        tags_text = str(payload.get('tags', ''))
        # 쉼표, 공백, 세미콜론 등 다양한 구분자를 기준으로 태그를 분리합니다.
        tag_list = [tag.strip() for tag in re.split(r'[,;\s]\s*', tags_text) if tag]

        for keyword in query_keywords:
            if keyword in tag_list:
                bonus += 0.5  # 태그가 정확히 일치하면 큰 보너스!

        # 2. 서비스명 필드: 중간 가중치 (+0.2)
        service_name = str(payload.get('서비스명', ''))
        for keyword in query_keywords:
            if keyword in service_name:
                bonus += 0.2

        # 3. 지원내용 필드: 낮은 가중치 (+0.05)
        support_content = str(payload.get('지원내용', ''))
        for keyword in query_keywords:
            if keyword in support_content:
                bonus += 0.05

        reranked_hits.append({'original_hit': hit, 'final_score': hit.score + bonus})

    # 'final_score'가 높은 순으로 다시 정렬
    reranked_hits.sort(key=lambda x: x['final_score'], reverse=True)

    # 3. 최종 결과 출력
    console = Console()
    table = Table(title=f"'{query_text}' 검색 결과 (Top {top_k})")
    table.add_column("Final Score", style="cyan")
    table.add_column("Original Score", style="dim cyan")
    table.add_column("서비스명", style="magenta")
    table.add_column("Tags", style="green") # Tags를 직접 확인하기 위해 추가

    for item in reranked_hits[:top_k]:
        hit = item['original_hit']
        payload = hit.payload
        table.add_row(
            f"{item['final_score']:.4f}",
            f"{hit.score:.4f}",
            str(payload.get('서비스명', 'N/A')),
            str(payload.get('tags', 'N/A'))
        )

    console.print(table)


# --- 검색 테스트 ---
search_services("우리 아이 학비 지원해주는 정책 알려줘")
search_services("사업 시작하는데 자금이 부족해")
search_services("나이 많은 어르신들을 위한 일자리 없을까?")

                             '우리 아이 학비 지원해주는 정책 알려줘' 검색 결과 (Top 3)                             
┏━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Final Score ┃ Original Score ┃ 서비스명                 ┃ Tags                                                  ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 0.8791      │ 0.6291         │ 유아학비 (누리과정) 지원 │ ['유아학비', '저소득층', '유치원', '교육비',          │
│             │                │                          │ '지원대상', '유아', '사립유치원']                     │
│ 0.6916      │ 0.6416         │ 초·중·고 교육비 지원     │ ['저소득층', '교육비지원', '학비', '학교급식비',      │
│             │                │                          │ '교육정보화지원', '방과후학교', '한부모가족']         │
│ 0.6307      │ 0.5307         │ 영유아보육료 지원        │ ['영유아보육료', '지원대상', '보육료지원',            │
│             │                │                          │ '아동지원', '유아교육', '보육서비스', '어린이집']     │
└─────────────┴────────────────┴──────────────────────────┴───────────────────────────────────────────────────────┘

                                 '사업 시작하는데 자금이 부족해' 검색 결과 (Top 3)                                 
┏━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Final Score ┃ Original Score ┃ 서비스명                  ┃ Tags                                                 ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 0.7185      │ 0.4685         │ 재창업기업 사업화 지원    │ ['창업지원', '재창업자', '대출', '멘토링',           │
│             │                │                           │ '사업화자금']                                        │
│ 0.7050      │ 0.4550         │ 협동화사업 자금 융자      │ ['중소기업', '자금 융자', '협동화', '대출',          │
│             │                │                           │ '융자지원', '자금조달', '대출금리']                  │
│ 0.6807      │ 0.4307         │ 산림사업 사업비 융자 지원 │ ['산림사업', '융자', '산림소유자', '지원', '사업비'] │
└─────────────┴────────────────┴───────────────────────────┴──────────────────────────────────────────────────────┘

                           '나이 많은 어르신들을 위한 일자리 없을까?' 검색 결과 (Top 3)                            
┏━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Final Score ┃ Original Score ┃ 서비스명                            ┃ Tags                                       ┃
┡━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 0.7161      │ 0.5161         │ 산림병해충예찰방제단 일자리 제공    │ ['청년', '저소득층', '일자리',             │
│             │                │                                     │ '산림병해충', '취업취약계층', '지급',      │
│             │                │                                     │ '교육']                                    │
│ 0.7019      │ 0.5019         │ 산촌생태마을 운영매니저 일자리 제공 │ ['청년', '주거안정', '창업지원',           │
│             │                │                                     │ '저소득층', '대출', '신혼부부', '장애인']  │
│ 0.6893      │ 0.4893         │ 산사태현장예방단 일자리 제공        │ ['청년', '저소득층', '일자리',             │
│             │                │                                     │ '산사태현장예방단', '취업취약계층',        │
│             │                │                                     │ '지역주민', '응급조치']                    │
└─────────────┴────────────────┴─────────────────────────────────────┴────────────────────────────────────────────┘